In [1]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.
        builder.
        appName("curso_pyspark").
        getOrCreate()
)

spark

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/08/01 19:13:42 WARN Utils: Your hostname, Matheuss-MacBook-Air.local, resolves to a loopback address: 127.0.0.1; using 192.168.0.135 instead (on interface en0)
25/08/01 19:13:42 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/08/01 19:14:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Básico de Pyspark

## Dataframes

<!-- ###
### create table (
###    nome VARCHAR(100) not null,
###    sobrenome VARCHAR(100) not null,
###    idade INT not null
###)

#  nome sobrenome idade
# (N1      N2       I1) -->

create table ( <br />
   nome VARCHAR(100) not null, <br />
   sobrenome VARCHAR(100) not null, <br />
   idade INT not null <br />
) <br />
<br /><br />
nome sobrenome idade
(N1      N2       I1)

In [2]:
from pyspark.sql.types import StructField, StructType, StringType, IntegerType

data = [
    ##  C1          C2        C3
    ("Matheus", "Cantarutti", 31),
    ("Ana", "Cláudia", 18),
    ("Brunno", "Oliveira", 25)
]

schema = StructType([
    StructField("Nome", StringType(), True),
    StructField("Sobre_Nome", StringType(), True),
    StructField("Idade", IntegerType(), True)
])

df = spark.createDataFrame(data, schema)
df.printSchema()

df.show()

root
 |-- Nome: string (nullable = true)
 |-- Sobre_Nome: string (nullable = true)
 |-- Idade: integer (nullable = true)



+-------+----------+-----+
|   Nome|Sobre_Nome|Idade|
+-------+----------+-----+
|Matheus|Cantarutti|   31|
|    Ana|   Cláudia|   18|
| Brunno|  Oliveira|   25|
+-------+----------+-----+



In [3]:
df.createOrReplaceTempView("pessoas")

In [4]:
### SQL --> Pyspark

spark.sql( 
'''    
    select
        *
    from pessoas
    where Idade < 20
'''
).show()

+----+----------+-----+
|Nome|Sobre_Nome|Idade|
+----+----------+-----+
| Ana|   Cláudia|   18|
+----+----------+-----+



In [5]:
from pyspark.sql import functions as F

# df.filter('Idade < 20').show()
df.filter(
    F.col('Idade') < 20
).show()

+----+----------+-----+
|Nome|Sobre_Nome|Idade|
+----+----------+-----+
| Ana|   Cláudia|   18|
+----+----------+-----+



## Tipo de Dados

- TIPO TEXTO/STRING >> abrangendo apenas as funções que tratam texto
- TIPO DATA (DATA ESTÁ COM O TIPO DE STRING) >> Converter o seu texto para Data

- FLOAT/DECIMAL e INTERGER

In [6]:
spark.sql('''
    select
        *,
        cast(Idade * 5 as string) as Idade_2
    from pessoas
''').printSchema()

root
 |-- Nome: string (nullable = true)
 |-- Sobre_Nome: string (nullable = true)
 |-- Idade: integer (nullable = true)
 |-- Idade_2: string (nullable = true)



In [7]:
(   
    # nome da coluna, expressões/funcao
    df.withColumn('Idade_2', F.col('Idade') * 5)
      .withColumn('data', F.lit('2025-01-01')) # current date
      .withColumn('data2', F.to_date(F.col('data'), 'yyyy-MM-dd'))
      .withColumn('Idade_3', F.expr('cast(Idade * 5 as string) as Idade_3'))

).show()

+-------+----------+-----+-------+----------+----------+-------+
|   Nome|Sobre_Nome|Idade|Idade_2|      data|     data2|Idade_3|
+-------+----------+-----+-------+----------+----------+-------+
|Matheus|Cantarutti|   31|    155|2025-01-01|2025-01-01|    155|
|    Ana|   Cláudia|   18|     90|2025-01-01|2025-01-01|     90|
| Brunno|  Oliveira|   25|    125|2025-01-01|2025-01-01|    125|
+-------+----------+-----+-------+----------+----------+-------+



## Cardinalidade

- Aula teórica explicativa sobre cardinalidade

### Dataframe de exemplo

In [8]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructField, StructType, StringType

spark = (
    SparkSession.
        builder.
        getOrCreate()
)
# CLOUD --> spark.sql('select * from bd.aulas.tabelas')
# df <- na leitura dos arquivos (.csv ou xlsx)
## spark.read

data = [
    ("1", "PAGO"),
    ("2", "APROVADO"),
    ("3", "RECUSADO"),
    ("4", "ENTREGUE"),
    ("5", "CANCELADO"),
    ("6", "NÃO ENTREGUE")
]

schema = StructType([
    StructField("cd_identificacao", StringType(), True),
    StructField("status", StringType(), True)
])

df = spark.createDataFrame(data, schema)
df.printSchema()
df.show()

root
 |-- cd_identificacao: string (nullable = true)
 |-- status: string (nullable = true)

+----------------+------------+
|cd_identificacao|      status|
+----------------+------------+
|               1|        PAGO|
|               2|    APROVADO|
|               3|    RECUSADO|
|               4|    ENTREGUE|
|               5|   CANCELADO|
|               6|NÃO ENTREGUE|
+----------------+------------+



# Lendo arquivos

## Command Separated Value (.csv) 

In [9]:
caminho = './dados/csv/'
cliente = 'clientes.csv'
status = 'status.csv'
pedidos = 'pedidos.csv'

clientes = (
    spark.read.csv(
        f'{caminho}{cliente}', 
        sep=';',
        header=True
    )
)

status = (
    spark.read.csv(
        f'{caminho}{status}', 
        sep=';',
        header=True
    )
)

pedidos = (
    spark.read.csv(
        f'{caminho}{pedidos}', 
        sep=';',
        header=True
    )
)

pedidos.show()

+---------+----------+--------------------+-------------+-----------+
|cd_pedido|cd_cliente|             produto|status pedido|      valor|
+---------+----------+--------------------+-------------+-----------+
|       10|         2|TV LED 55''- SANSUNG|            2|R$ 5.500,48|
|       12|         3|MÁQUINA DE LAVAR ...|            4|R$ 4.850,00|
|       13|         5|NOTEBOOK LENOVO - i5|            3|R$ 3.585,50|
|       15|         7|CELULAR POCO M4 -...|            1|R$ 1.285,00|
|       12|         2|MÁQUINA DE LAVAR ...|            4|R$ 4.850,00|
+---------+----------+--------------------+-------------+-----------+



## Excel

In [10]:
import pandas as pd

caminho = './dados/xlsx/'
cliente = 'clientes.xlsx'
aba = 'clientes'

def ler_excel(file_path, aba):
    try:
        df = pd.read_excel(file_path, sheet_name=aba, engine='openpyxl')
    except FileNotFoundError:
        print(f"Arquivo {file_path} não encontrado.")
    except ValueError as e:
        print(f"Erro ao ler a aba de nome {aba}: {e}")
    except Exception as e:
        print(f"Ocorreu um erro inesperado: {e}")
    return df

df = ler_excel(f'{caminho}{cliente}', aba)
df = spark.createDataFrame(df)
print(type(df))

<class 'pyspark.sql.classic.dataframe.DataFrame'>


# Intermediário

## Filtros

### Filter

In [17]:
caminho = './dados/csv/'
cliente = 'clientes.csv'

clientes = (
    spark.read.csv(
        f'{caminho}{cliente}', 
        sep=';',
        header=True
    )
)

clientes.show()
clientes.createOrReplaceTempView("clientes")

+----------+--------------------+----+---------------+
|cd_cliente|        nome_cliente|sexo|data_nascimento|
+----------+--------------------+----+---------------+
|         1|      MARIA DO CARMO|   F|     1963-02-10|
|         2|      LUCAS DA SILVA|   M|     1998-07-25|
|         3| SILVÉRIO DA FONSECA|   M|     1975-01-15|
|         4|    BRUNNO FERNANDES|   M|     2004-04-05|
|         5|FERNANDA DO NASCI...|   F|     1994-09-12|
|         6|  CARLOS DE OLIVEIRA|   M|     1998-04-15|
|         7|  VITÓRIA DE ALMEIDA|   F|     1994-06-12|
|         8|   GABRIELA DE SILVA|   F|     1994-03-27|
|         9|      MARCOS PACHECO|   M|     1989-02-02|
|        10|   DANIEL WANDERGAST|   M|     1980-01-24|
+----------+--------------------+----+---------------+



In [18]:
spark.sql('''
    select 
        * 
    from clientes
    where sexo = 'F'
''').show()

+----------+--------------------+----+---------------+
|cd_cliente|        nome_cliente|sexo|data_nascimento|
+----------+--------------------+----+---------------+
|         1|      MARIA DO CARMO|   F|     1963-02-10|
|         5|FERNANDA DO NASCI...|   F|     1994-09-12|
|         7|  VITÓRIA DE ALMEIDA|   F|     1994-06-12|
|         8|   GABRIELA DE SILVA|   F|     1994-03-27|
+----------+--------------------+----+---------------+



In [19]:
clientes2 = (
    # para filtrar dados, o filter é a função utilizada
    clientes
        .filter(
            F.col('sexo') == 'F'
        )
)
clientes2.show()

+----------+--------------------+----+---------------+
|cd_cliente|        nome_cliente|sexo|data_nascimento|
+----------+--------------------+----+---------------+
|         1|      MARIA DO CARMO|   F|     1963-02-10|
|         5|FERNANDA DO NASCI...|   F|     1994-09-12|
|         7|  VITÓRIA DE ALMEIDA|   F|     1994-06-12|
|         8|   GABRIELA DE SILVA|   F|     1994-03-27|
+----------+--------------------+----+---------------+



In [20]:
clientes3 = (
    # para filtrar dados, o filter é a função utilizada
    clientes
        .filter(
            F.col('sexo') == 'M'
        )
)
clientes3.show()

+----------+-------------------+----+---------------+
|cd_cliente|       nome_cliente|sexo|data_nascimento|
+----------+-------------------+----+---------------+
|         2|     LUCAS DA SILVA|   M|     1998-07-25|
|         3|SILVÉRIO DA FONSECA|   M|     1975-01-15|
|         4|   BRUNNO FERNANDES|   M|     2004-04-05|
|         6| CARLOS DE OLIVEIRA|   M|     1998-04-15|
|         9|     MARCOS PACHECO|   M|     1989-02-02|
|        10|  DANIEL WANDERGAST|   M|     1980-01-24|
+----------+-------------------+----+---------------+



In [ ]:
clientes4 = (
    # para filtrar dados, o filter é a função utilizada
    clientes
        .filter(
            ~ (F.col('sexo') == 'F')
        )
)
clientes4.show()

+----------+-------------------+----+---------------+
|cd_cliente|       nome_cliente|sexo|data_nascimento|
+----------+-------------------+----+---------------+
|         2|     LUCAS DA SILVA|   M|     1998-07-25|
|         3|SILVÉRIO DA FONSECA|   M|     1975-01-15|
|         4|   BRUNNO FERNANDES|   M|     2004-04-05|
|         6| CARLOS DE OLIVEIRA|   M|     1998-04-15|
|         9|     MARCOS PACHECO|   M|     1989-02-02|
|        10|  DANIEL WANDERGAST|   M|     1980-01-24|
+----------+-------------------+----+---------------+



### Isin

In [28]:
spark.sql('''
    select
        *
    from clientes
    where cd_cliente in ('2', '3')
''').show()

+----------+-------------------+----+---------------+
|cd_cliente|       nome_cliente|sexo|data_nascimento|
+----------+-------------------+----+---------------+
|         2|     LUCAS DA SILVA|   M|     1998-07-25|
|         3|SILVÉRIO DA FONSECA|   M|     1975-01-15|
+----------+-------------------+----+---------------+



In [29]:
spark.sql('''
    select
        *
    from clientes
    where cd_cliente not in ('2', '3')
''').show()

+----------+--------------------+----+---------------+
|cd_cliente|        nome_cliente|sexo|data_nascimento|
+----------+--------------------+----+---------------+
|         1|      MARIA DO CARMO|   F|     1963-02-10|
|         4|    BRUNNO FERNANDES|   M|     2004-04-05|
|         5|FERNANDA DO NASCI...|   F|     1994-09-12|
|         6|  CARLOS DE OLIVEIRA|   M|     1998-04-15|
|         7|  VITÓRIA DE ALMEIDA|   F|     1994-06-12|
|         8|   GABRIELA DE SILVA|   F|     1994-03-27|
|         9|      MARCOS PACHECO|   M|     1989-02-02|
|        10|   DANIEL WANDERGAST|   M|     1980-01-24|
+----------+--------------------+----+---------------+



In [23]:
clientes.filter(
    F.col('cd_cliente').isin(['2', '3'])
).show()

+----------+-------------------+----+---------------+
|cd_cliente|       nome_cliente|sexo|data_nascimento|
+----------+-------------------+----+---------------+
|         2|     LUCAS DA SILVA|   M|     1998-07-25|
|         3|SILVÉRIO DA FONSECA|   M|     1975-01-15|
+----------+-------------------+----+---------------+



In [24]:
clientes.filter(
    ~(F.col('cd_cliente').isin(['2', '3']))
).show()

+----------+--------------------+----+---------------+
|cd_cliente|        nome_cliente|sexo|data_nascimento|
+----------+--------------------+----+---------------+
|         1|      MARIA DO CARMO|   F|     1963-02-10|
|         4|    BRUNNO FERNANDES|   M|     2004-04-05|
|         5|FERNANDA DO NASCI...|   F|     1994-09-12|
|         6|  CARLOS DE OLIVEIRA|   M|     1998-04-15|
|         7|  VITÓRIA DE ALMEIDA|   F|     1994-06-12|
|         8|   GABRIELA DE SILVA|   F|     1994-03-27|
|         9|      MARCOS PACHECO|   M|     1989-02-02|
|        10|   DANIEL WANDERGAST|   M|     1980-01-24|
+----------+--------------------+----+---------------+

